In [1]:
import re
import csv
import numpy as np
import yahoo_finance
from yahoo_finance import Share
from sklearn import linear_model
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import time
from datetime import datetime, timedelta

In [2]:

class StockPredict:
    
    #initialize the object
    def __init__(self,symbol):
        self.symbol = symbol
        self.filename = symbol +".csv"
        self.stock = Share(self.symbol)
     
    #compute start, end & predict dates
    def dateCalc(self):
        import time
        self.start_date = datetime.now() - timedelta(days=30)
        self.start_date = (self.start_date.strftime("%Y-%m-%d")) #date format: yyyy-mm-dd
        self.end_date = (time.strftime("%Y-%m-%d"))  #date format: yyyy-mm-dd
        N = 1
        self.predict_date= datetime.now() + timedelta(days=N)
        self.prediction_date = re.findall(r"\b([0-9]{1,4})[-]([0-9]{1,2})[-]([0-9]{2})\b",str(self.predict_date))
        self.prediction_date = int(self.prediction_date[0][2]) #date format: dd
        return
     
    #Obtain historical stock data
    def access_stockinfo(self):
        self.storedata = (self.stock.get_historical(self.start_date, self.end_date))
        return
    
    #Store historical stock data in CSV file
    def store_stockinfo(self):
        keys = self.storedata[0].keys()
        with open(self.filename, 'wb') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(self.storedata)
        return
    
    #Compute list of dates
    def get_stockdate(self):
        with open(self.filename,'r') as csvfile:
            csvFileReader = csv.reader(csvfile)
            next(csvFileReader) #skipping column names
            global dates
            dates= [re.findall(r"\b([0-9]{1,4})[-]([0-9]{1,2})[-]([0-9]{2})\b",row[5]) for row in csvFileReader]
            dates =[int(date[0][2]) for date in dates]
        return
    
    #Compute list of price
    def get_stockprice(self):
        with open(self.filename,'r') as csvfile:
            csvFileReader = csv.reader(csvfile)
            next(csvFileReader) #skipping column names
            global prices
            prices = [float(row[2]) for row in csvFileReader]
            return
    
    #Calculate predicted price using linear regression model
    def predict_stockprice(self,dates,prices,predictDate):
        linear_mod = linear_model.LinearRegression() #defining the linear regression model
        dates = np.reshape(dates,(len(dates),1)) # converting to matrix of n X 1
        prices = np.reshape(prices,(len(prices),1))
        linear_mod.fit(dates,prices) #fitting the data points in the model
        predicted_price =linear_mod.predict(predictDate)
        return predicted_price[0][0]

    #Plotting dataset onto the graph
    def show_stockplot(self,dates,prices):
        linear_mod = linear_model.LinearRegression()
        dates = np.reshape(dates,(len(dates),1)) # converting to matrix of n X 1
        prices = np.reshape(prices,(len(prices),1))
        linear_mod.fit(dates,prices) #fitting the data points in the model
        plt.scatter(dates,prices,color='red') #plotting the initial datapoints 
        plt.plot(dates,linear_mod.predict(dates),color='blue',linewidth=4) #plotting the line made by linear regression
        plt.show()
        return
 
#Get input from user 
symbol = raw_input("Enter symbol for stock prediction: ")

#create a StockPredict class instance
stockObj = StockPredict(symbol.upper())
#Check for valid stock entry
if not stockObj.stock.get_price():
    print "Invalid Stock! Please enter correct Symbol."
else:
    stockObj.dateCalc()
    stockObj.access_stockinfo()
    stockObj.store_stockinfo()
    stockObj.get_stockdate()
    stockObj.get_stockprice()
    
    #display dataset
    df = pd.read_csv(stockObj.filename)
    print " "
    print "Historical data of " + symbol.upper() + " from " + stockObj.start_date + " to " + stockObj.end_date
    print df

    predicted_price = stockObj.predict_stockprice(dates,prices,stockObj.prediction_date)  
    print "The predicted stock open price for " + symbol.upper() + " on "  + str(stockObj.predict_date.strftime("%d-%m-%Y")) + " is: $",str(predicted_price)
    stockObj.show_stockplot(dates,prices) 

Enter symbol for stock prediction: csco
 
Historical data of CSCO from 2017-03-05 to 2017-04-04
      Volume Symbol  Adj_Close       High        Low        Date      Close  \
0   18022200   CSCO  33.410000  33.419998  32.930000  2017-04-04  33.410000   
1   19367800   CSCO  33.290001  33.939999  33.459999  2017-04-03  33.580002   
2   14038500   CSCO  33.508098  33.880001  33.660000  2017-03-31  33.799999   
3   12801600   CSCO  33.448619  33.930000  33.720001  2017-03-30  33.740002   
4   16504100   CSCO  33.448619  34.049999  33.650002  2017-03-29  33.740002   
5   21179300   CSCO  33.726200  34.209999  33.820000  2017-03-28  34.020000   
6   17740600   CSCO  33.696460  34.029999  33.750000  2017-03-27  33.990002   
7   18125100   CSCO  33.785683  34.299999  33.950001  2017-03-24  34.080002   
8   16709300   CSCO  33.666717  34.259998  33.910000  2017-03-23  33.959999   
9   16646100   CSCO  33.805507  34.200001  33.779999  2017-03-22  34.099998   
10  21406000   CSCO  33.587409  34.